In [ ]:
# --- Setup (run if needed) ---
!pip install -U langchain langchain-google-genai

import os
from getpass import getpass
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass("Enter your Google Gemini API key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.4)
result  = {}
# --- Step 1: Topic generator ---
topic_prompt = PromptTemplate.from_template(
    "Given the broad domain: {domain}\n"
    "Propose one specific, fresh **niche blog topic** (max 12 words). Return only the topic text."
)

topic_chain = topic_prompt | llm | StrOutputParser()

# (Optional quick test)
# print(topic_chain.invoke({"domain": "Productivity"}))


Enter your Google Gemini API key: ··········
Productivity for neurodivergent individuals navigating sensory overload.


In [ ]:
from langchain_core.runnables import RunnableLambda

# Bridge: str -> {"topic": str}
to_topic_dict = RunnableLambda(lambda s: {"topic": s})

# Title prompt & chain
title_prompt = PromptTemplate.from_template(
    "Given this blog topic: {topic}\n"
    "Write a catchy blog **title** (≤ 12 words). Return only the title."
)
title_chain = title_prompt | llm | StrOutputParser()

# Compose: topic -> (bridge) -> title
# topic_to_title = topic_chain | to_topic_dict | title_chain

# print(topic_to_title.invoke({"domain": "Productivity"}))



Your Body Clock: The Ultimate Productivity Hack.


In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from operator import itemgetter

# Bridge: str -> {"title": str}
to_title_dict = RunnableLambda(lambda s: {"title": s})

# Summary prompt & chain
summary_prompt = PromptTemplate.from_template(
    "Based on this blog title: {title}\n"
    "Write a single-paragraph blog summary (4–6 sentences). Return only the paragraph."
)
summary_chain = summary_prompt | llm | StrOutputParser()

# Compose full 3-step flow: domain -> topic -> title -> summary
domain_to_summary = (
    topic_chain          # needs {"domain": "..."} → str(topic)
    | to_topic_dict      # str(topic) → {"topic": str}
    | title_chain        # → str(title)
    | to_title_dict      # str(title) → {"title": str}
    | summary_chain      # → str(summary)
)
# Assume you already ran `result = full_pipeline.invoke({"domain": "Productivity"})`
# result -> {'domain': '...', 'topic': '...', 'title': '...', 'summary': '...'}



# --- Wrap dict in a Runnable so .invoke works
full_pipeline = RunnableParallel({
    "domain": RunnablePassthrough() | RunnableLambda(itemgetter("domain")),
    "topic":  topic_chain,  # expects {"domain"}
    "title":  topic_chain | to_topic_dict | title_chain,
    "summary": topic_chain | to_topic_dict | title_chain | to_title_dict | summary_chain,
})


result = full_pipeline.invoke({"domain": "Productivity"})
# print(result)


# response = domain_to_summary.invoke({"domain": "Productivity"})



In [ ]:
def to_markdown(res: dict) -> str:
    return f"""# Blog Draft

**Domain:** {res['domain']}

## {res['title']}

### Topic
{res['topic']}

### Summary
{res['summary']}
"""

md = to_markdown(result)
print(md)


# Blog Draft

**Domain:** Productivity

## Why 'Unproductive' Time Is Your Creative Superpower

### Topic
Productivity for neurodivergent minds: custom systems, not generic hacks.

### Summary
This blog challenges conventional productivity models, offering a refreshing perspective tailored for neurodivergent individuals. It recognizes that standard approaches often clash with neurodivergent brains, leading to burnout and frustration rather than efficiency. Instead, the post advocates for a deeply personalized approach, empowering readers to understand their unique cognitive strengths and challenges. It encourages crafting bespoke productivity systems that honor individual needs, energy fluctuations, and processing styles, rather than forcing conformity. Ultimately, "Neurodivergent Productivity" aims to help readers build sustainable, effective strategies that genuinely work *with* their brains, fostering success on their own terms.

